# Advanced Control Architectures

Building on the simple feedback loop from the previous notebook, we can now explore more sophisticated and powerful control architectures. The agent-based, message-driven design of the platform makes it easy to create complex systems by composing simple agents.

This notebook covers two advanced patterns:
1.  **Hierarchical Control**: A high-level, supervisory agent that commands a low-level control agent.
2.  **Decentralized Control**: Multiple, independent control agents operating in parallel within a larger network.

## Part 1: Hierarchical Control

In a hierarchical system, a high-level agent acts as a 'supervisor', making strategic decisions based on the overall system state. It then sends commands to low-level agents that are responsible for the direct, real-time control of physical components.

### The Scenario
We will model a reservoir where a `LocalControlAgent` (LCA) is trying to maintain a 'normal' water level setpoint. A supervisory `CentralDispatcher` agent also monitors the water level. If the level exceeds a 'flood' threshold, the dispatcher intervenes, sending a new, lower setpoint to the LCA to quickly reduce the water level.

### Architecture
The key addition to our previous architecture is the `CentralDispatcher` and a new communication topic for commands.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.DiGraph()
G.add_node("Reservoir", type='physical')
G.add_node("Gate", type='physical')
G.add_node("Twin Agent", type='agent')
G.add_node("LCA (PID)", type='agent')
G.add_node("Dispatcher", type='agent')

G.add_edge("Reservoir", "Twin Agent", label='reads state')
G.add_edge("Twin Agent", "LCA (PID)", label='pub: state')
G.add_edge("Twin Agent", "Dispatcher", label='pub: state')
G.add_edge("Dispatcher", "LCA (PID)", label='pub: command\n(new setpoint)')
G.add_edge("LCA (PID)", "Gate", label='pub: action')

node_colors = ['lightblue' if G.nodes[n]['type'] == 'physical' else 'lightgreen' for n in G.nodes]
pos = nx.spring_layout(G, seed=42)

plt.figure(figsize=(10, 8))
nx.draw(G, pos, with_labels=True, node_size=4000, node_color=node_colors, font_size=10, arrowsize=20)
edge_labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red')
plt.title('Hierarchical Control Architecture')
plt.show()

In [ ]:
from swp.core_engine.testing.simulation_harness import SimulationHarness
from swp.central_coordination.collaboration.message_bus import MessageBus
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.local_agents.control.pid_controller import PIDController
from swp.local_agents.control.local_control_agent import LocalControlAgent
from swp.local_agents.perception.digital_twin_agent import DigitalTwinAgent
from swp.central_coordination.dispatch.central_dispatcher import CentralDispatcher
import pandas as pd

# Setup function from examples, included for clarity
def setup_hierarchical_control_system(message_bus, simulation_dt):
    RESERVOIR_STATE_TOPIC = "state.reservoir.level"
    GATE_ACTION_TOPIC = "action.gate.opening"
    GATE_COMMAND_TOPIC = "command.gate1.setpoint"
    gate = Gate(name="gate_1", initial_state={'opening': 0.1}, parameters={'width': 10}, message_bus=message_bus, action_topic=GATE_ACTION_TOPIC)
    reservoir = Reservoir(name="reservoir_1", initial_state={'volume': 28.5e6, 'water_level': 19.0}, parameters={'surface_area': 1.5e6})
    reservoir_twin = DigitalTwinAgent(agent_id="twin_res1", simulated_object=reservoir, message_bus=message_bus, state_topic=RESERVOIR_STATE_TOPIC)
    pid = PIDController(Kp=-0.8, Ki=-0.1, Kd=-0.2, setpoint=15.0)
    lca = LocalControlAgent(agent_id="lca_g1", controller=pid, message_bus=message_bus, observation_topic=RESERVOIR_STATE_TOPIC, observation_key='water_level', action_topic=GATE_ACTION_TOPIC, dt=simulation_dt, command_topic=GATE_COMMAND_TOPIC)
    rules = {'flood_threshold': 18.0, 'normal_setpoint': 15.0, 'flood_setpoint': 12.0}
    dispatcher = CentralDispatcher(agent_id="dispatcher1", message_bus=message_bus, state_subscriptions={'reservoir_level': RESERVOIR_STATE_TOPIC}, command_topics={'gate1_command': GATE_COMMAND_TOPIC}, rules=rules)
    return [reservoir, gate], [reservoir_twin, lca, dispatcher]

# Main simulation logic
harness = SimulationHarness(config={'duration': 500, 'dt': 1.0})
components, agents = setup_hierarchical_control_system(harness.message_bus, 1.0)
for component in components: harness.add_component(component)
for agent in agents: harness.add_agent(agent)
harness.add_connection("reservoir_1", "gate_1")

import sys
original_stdout = sys.stdout
with open('simulation_log.txt', 'w') as f:
    sys.stdout = f
    harness.build()
    harness.run_mas_simulation()
sys.stdout = original_stdout

print("Hierarchical simulation complete.")

#### Results
The simulation starts with the water level at 19.0m, which is above the `flood_threshold` of 18.0m. The `CentralDispatcher` immediately publishes a new, lower setpoint of 12.0m to the LCA. The plot clearly shows the LCA working to bring the water level down to this new, emergency setpoint.

In [ ]:
df = pd.DataFrame(harness.history)
time = df['time']
res_level = df['reservoir_1'].apply(lambda x: x['water_level'])
gate_opening = df['gate_1'].apply(lambda x: x['opening'])

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
ax1.plot(time, res_level, label='Reservoir Water Level')
ax1.axhline(y=15.0, color='g', linestyle='--', label='Original Setpoint (15.0m)')
ax1.axhline(y=12.0, color='r', linestyle='--', label='Emergency Setpoint (12.0m)')
ax1.set_ylabel('Water Level (m)')
ax1.set_title('Hierarchical Control Performance')
ax1.grid(True)
ax1.legend()

ax2.plot(time, gate_opening, label='Gate Opening', color='purple')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Gate Opening (%)')
ax2.grid(True)
ax2.legend()
plt.show()

---

## Part 2: Decentralized Control

Decentralized control involves multiple, independent agents operating in parallel to control different parts of a larger system. This is common in real-world networks where, for example, each dam operator manages their own facility, but their actions affect others downstream.

### The Scenario
We will model a branched river network. Two reservoirs, each with its own gate and dedicated `LocalControlAgent`, release water into separate channels. These channels then merge into a single main channel. Each LCA operates independently to maintain the setpoint in its own reservoir.

### Architecture & Code
The code below sets up this branched network, including two independent sets of `(DigitalTwinAgent, LocalControlAgent)` pairs. The `networkx` graph clearly shows the two parallel control loops.

In [ ]:
from swp.simulation_identification.physical_objects.river_channel import RiverChannel

# 1. Setup
harness = SimulationHarness(config={'duration': 1000, 'dt': 1.0})
message_bus = harness.message_bus

# 2. Physical Components
res1 = Reservoir(name="res1", initial_state={'water_level': 10.0}, parameters={'surface_area': 1.5e6})
g1 = Gate(name="g1", initial_state={'opening': 0.1}, parameters={'width': 10}, message_bus=message_bus, action_topic="action.g1.opening")
trib_chan = RiverChannel(name="trib_chan", initial_state={'volume': 2e5}, parameters={'k': 0.0002})
res2 = Reservoir(name="res2", initial_state={'water_level': 20.0}, parameters={'surface_area': 1.5e6})
g2 = Gate(name="g2", initial_state={'opening': 0.1}, parameters={'width': 15}, message_bus=message_bus, action_topic="action.g2.opening")
main_chan = RiverChannel(name="main_chan", initial_state={'volume': 8e5}, parameters={'k': 0.0001})
g3 = Gate(name="g3", initial_state={'opening': 0.5}, parameters={'width': 20})
for comp in [res1, g1, trib_chan, res2, g2, main_chan, g3]: harness.add_component(comp)
harness.add_connection("res1", "g1"); harness.add_connection("g1", "trib_chan"); harness.add_connection("res2", "g2");
harness.add_connection("trib_chan", "main_chan"); harness.add_connection("g2", "main_chan"); harness.add_connection("main_chan", "g3");

# 3. Agent Systems (Two independent loops)
twin1 = DigitalTwinAgent(agent_id="twin1", simulated_object=res1, message_bus=message_bus, state_topic="state.res1.level")
pid1 = PIDController(Kp=-0.5, Ki=-0.05, Kd=-0.1, setpoint=12.0)
lca1 = LocalControlAgent(agent_id="lca1", controller=pid1, message_bus=message_bus, observation_topic="state.res1.level", observation_key="water_level", action_topic="action.g1.opening", dt=1.0)

twin2 = DigitalTwinAgent(agent_id="twin2", simulated_object=res2, message_bus=message_bus, state_topic="state.res2.level")
pid2 = PIDController(Kp=-0.4, Ki=-0.04, Kd=-0.1, setpoint=18.0)
lca2 = LocalControlAgent(agent_id="lca2", controller=pid2, message_bus=message_bus, observation_topic="state.res2.level", observation_key="water_level", action_topic="action.g2.opening", dt=1.0)
for agent in [twin1, lca1, twin2, lca2]: harness.add_agent(agent)

# 4. Build and Run
import sys; original_stdout = sys.stdout
with open('simulation_log.txt', 'w') as f:
    sys.stdout = f; harness.build(); harness.run_mas_simulation()
sys.stdout = original_stdout
print("Decentralized simulation complete.")

# 5. Visualize Topology
G = nx.DiGraph(harness.topology)
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G, seed=42, iterations=100)
nx.draw(G, pos, with_labels=True, node_size=3000, node_color='lightblue', font_size=10, arrowsize=20)
plt.title('Decentralized Branched Network Topology')
plt.show()

#### Results
The plot shows the water levels of both reservoirs. Each one is successfully driven towards its own, independent setpoint by its dedicated control agent.

In [ ]:
df = pd.DataFrame(harness.history)
time = df['time']
res1_level = df['res1'].apply(lambda x: x['water_level'])
res2_level = df['res2'].apply(lambda x: x['water_level'])

plt.figure(figsize=(12, 6))
plt.plot(time, res1_level, label='Reservoir 1 Water Level')
plt.axhline(y=12.0, color='blue', linestyle='--', label='Reservoir 1 Setpoint (12.0m)')
plt.plot(time, res2_level, label='Reservoir 2 Water Level')
plt.axhline(y=18.0, color='orange', linestyle='--', label='Reservoir 2 Setpoint (18.0m)')
plt.ylabel('Water Level (m)')
plt.xlabel('Time (s)')
plt.title('Decentralized Control Performance')
plt.grid(True)
plt.legend()
plt.show()